In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.9 MB/s eta 0:00:00


In [ ]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('all_data.csv', delimiter=',')

In [ ]:
train.shape

(836, 4)

In [ ]:
train.head()

,review_id,app_id,review_text,label
0,138091990,2311190,1000000000000000000000000000000000000000000000...,0
1,138255165,1159690,A Raft style survival where you basically brin...,0
2,138255297,1159690,real good game,1
3,138252244,1667180,Finished it in 24 hours. I liked it. Somehow i...,1
4,138273841,1559600,Pretty horses game. addicted. ahh.,1


In [ ]:
print(train)

                     review_id,app_id,review_text,label
0     138091990,2311190,1000000000000000000000000000...
1     138255165,1159690,"A Raft style survival where...
2     Resource gathering is a chore and every single...
3     The items are very annoying to handle, fuel is...
4     The UI and crafting is terrible, everything yo...
...                                                 ...
2377  I hope the devs adds more content (modes, dail...
2378  Thank you devs for the chill feel, hope the ga...
2379  138129018,2233980,"Beta testing the game has b...
2380  138128927,1940340,"It's a blast. My only gripe...
2381  If you bought a game with the ""roguelite"" ta...

[2382 rows x 1 columns]


In [ ]:
train['label'].unique()

array([0, 1])

In [ ]:
train.describe()

,review_id,app_id,label
count,8.360000e+02,8.360000e+02,836.000000
mean,1.380582e+08,1.771113e+06,0.745215
std,1.304368e+06,4.753285e+05,0.436001
min,1.258393e+08,2.401000e+04,0.000000
25%,1.380971e+08,1.494420e+06,0.000000
50%,1.382485e+08,1.742020e+06,1.000000
75%,1.382780e+08,2.233990e+06,1.000000
max,1.383745e+08,2.407920e+06,1.000000


In [ ]:
new_df = train[['review_text', 'label']]

In [ ]:
new_df.head()

,review_text,label
0,1000000000000000000000000000000000000000000000...,0
1,A Raft style survival where you basically brin...,0
2,real good game,1
3,Finished it in 24 hours. I liked it. Somehow i...,1
4,Pretty horses game. addicted. ahh.,1


In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
# EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

In [ ]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['review_text']
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
train_size = 0.8
train_data=new_df.sample(frac=train_size,random_state=200)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (836, 2)
TRAIN Dataset: (669, 2)
TEST Dataset: (167, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 5)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = RobertaClass()
model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [ ]:
EPOCHS = 10
for epoch in range(EPOCHS):
    train(epoch)

In [ ]:
import torch
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

def calculate_metrics(preds, targets):
    metrics = {}
    metrics['accuracy'] = (preds == targets).sum().item()
    metrics['false_positives'] = ((preds == 1) & (targets == 0)).sum().item()
    metrics['false_negatives'] = ((preds == 0) & (targets == 1)).sum().item()
    return metrics

def valid(model, testing_loader):
    model.eval()
    n_correct = 0
    tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    print_step = 5000  # Set the desired print step here
    print_counter = 0

    # Create empty lists to store predicted labels and true labels
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()

            _, predicted_labels = torch.max(outputs.data, dim=1)
            n_correct += calculate_metrics(predicted_labels, targets)['accuracy']

            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)
            print_counter += 1

            # Append predicted labels and true labels to the respective lists
            all_predictions.extend(predicted_labels.cpu().numpy().tolist())
            all_targets.extend(targets.cpu().numpy().tolist())

            if print_counter % print_step == 0:
                loss_step = tr_loss / nb_tr_steps
                accu_step = (n_correct * 100) / nb_tr_examples
                print(f"Validation Loss per {print_step} steps: {loss_step}")
                print(f"Validation Accuracy per {print_step} steps: {accu_step}")

    epoch_loss = tr_loss / nb_tr_steps
    epoch_accu = (n_correct * 100) / nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    # Convert the lists to numpy arrays
    predictions = np.array(all_predictions)
    targets = np.array(all_targets)

    # Calculate the confusion matrix
    conf_matrix = confusion_matrix(targets, predictions)

    # Calculate precision, recall, and F1 scores
    class_names = ['Class 0', 'Class 1']  # Replace with your actual class names
    report = classification_report(targets, predictions, target_names=class_names, output_dict=True)

    # Printing the confusion matrix
    print("Confusion Matrix:")
    print(conf_matrix)

    # Printing precision, recall, and F1 scores
    print("Classification Report:")
    for class_name in class_names:
        print(f"{class_name}:")
        print(f"  Precision: {report[class_name]['precision']:.2f}")
        print(f"  Recall: {report[class_name]['recall']:.2f}")
        print(f"  F1 Score: {report[class_name]['f1-score']:.2f}")

    return epoch_accu


In [ ]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
42it [00:02, 14.58it/s]

Validation Loss Epoch: 0.4693733251437412
Validation Accuracy Epoch: 90.41916167664671
Confusion Matrix:
[[ 32  10]
 [  6 119]]
Classification Report:
Class 0:
  Precision: 0.84
  Recall: 0.76
  F1 Score: 0.80
Class 1:
  Precision: 0.92
  Recall: 0.95
  F1 Score: 0.94
Accuracy on test data = 90.42%


In [ ]:
output_model_file = 'pytorch_roberta_sentiment.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')
print('This tutorial is completed')

All files saved
This tutorial is completed
